In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression,\
                                RidgeCV, LassoCV, LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.decomposition import PCA
import eli5

### Functions

In [2]:
results = pd.DataFrame(columns = ['Model', 'Train Score', 'Val Score', 'X Val Score', 'RMSE Train', 'RMSE Val'])

def update_df(model, model_name):
    '''fn updates a dataframe for quick reference of R squared scores and RMSE'''
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        
        train_score = model.score(Xs_train, y_train)
        val_score = model.score(Xs_test, y_test)
        x_val_score = cross_val_score(model, Xs_train, y_train).mean()
        rmse1 = mean_squared_error(y_train, lr.predict(Xs_train))**0.5
        rmse2 = mean_squared_error(y_test, lr.predict(Xs_test))**0.5
        
        results.loc[len(results.index)] = [model_name, train_score, val_score, x_val_score, rmse1, rmse2] 
        
    return results

#### Scope

Using the data from The China Study we aim to create a predictive model for mortality rates/chances based on the various features collected by the study. Shy of a predictive model, we hope to glean any insights related to health and exume any correlations.

### Read in data

In [3]:
df = pd.read_csv('./cleaned_data_descriptive/combined_df.csv',index_col=[0])

In [4]:
df.head()

,county,sex,xiang,q010,q015,q016,q022,q023,q024,q025,...,u024,u025,u026,u027,u028,u029,u030,u031,u032,u033
0,AA,F,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AA,F,2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AA,F,3,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AA,T,3,40.2,60.0,70.0,3.3,22.4,41.9,35.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AB,F,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.shape

(275, 296)

In [6]:
df[df['sex'] == 'T'].shape

(69, 296)

In [7]:
ss = StandardScaler()

In [8]:
# df.columns.to_list() # sanity check

The data is separated by county -> sex -> xiang (village or small community)

Since a great majority of the data is missing M and F specific values, we can only rely on sex = 'T' which is an average of male and female according to the documents. 'xiang' is also split into 1, 2, and 3 where 3 is the average of 1 and 2. Like the 'sex' column we also do not have the more granular breakdowns for xiang 1 and 2. Thus we must rely on 'sex' = 'T' and 'xiang' == 3 for our dataset. 

In [9]:
clean_df = df[(df['sex'] == 'T') & (df['xiang'] == 3)]\
.dropna(subset='m005_ALL35_69').dropna(axis = 'columns')

In [10]:
clean_df['m005_ALL35_69']

3      11.29
7      11.82
11     10.55
15     16.19
19     13.59
       ...  
259    17.05
263    13.29
266    15.63
270    18.88
274    17.95
Name: m005_ALL35_69, Length: 66, dtype: float64

In [11]:
clean_df.shape

(66, 201)

### Linear Regression

Linear Regression makes the most sense. All of our data is continous and numeric albeit some on different numeric scales. We can use standard scaler and go from there.

In [12]:
X = clean_df.drop(columns = ['county', 'sex', 'xiang','m005_ALL35_69'])
y = clean_df['m005_ALL35_69']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33)

We're suffering from dimensionality here; we have too many features. We need to figure out which ones to focus on; trying to limit to 10 features for our 69 or so rows. Lasso and/or Ridge could help us out here.

In [13]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((49, 197), (17, 197), (49,), (17,))

Standard scaler to get everything on a comparable scale.

In [14]:
Xs_train = ss.fit_transform(X_train)
Xs_test = ss.transform(X_test)

In [15]:
lr = LinearRegression()

In [16]:
lr.fit(Xs_train, y_train)

LinearRegression()

In [17]:
update_df(lr, 'm005_ALL_35-69.LinReg')

,Model,Train Score,Val Score,X Val Score,RMSE Train,RMSE Val
0,m005_ALL_35-69.LinReg,1.0,0.88294,0.616316,8.404957e-15,1.206978


Basic regression model is overfit. Diving into Ridge and Lasso to see if we can address it. We suspect there may be data leakage between the mortality columns.

In [18]:
ridge = RidgeCV(alphas = np.logspace(0,2,100)) # 

In [19]:
lasso = LassoCV(alphas = np.arange(0.001, 10, 1))

In [20]:
ridge.fit(Xs_train, y_train)

RidgeCV(alphas=array([  1.        ,   1.04761575,   1.09749877,   1.149757  ,
         1.20450354,   1.26185688,   1.32194115,   1.38488637,
         1.45082878,   1.51991108,   1.59228279,   1.66810054,
         1.7475284 ,   1.83073828,   1.91791026,   2.009233  ,
         2.10490414,   2.20513074,   2.3101297 ,   2.42012826,
         2.53536449,   2.65608778,   2.7825594 ,   2.91505306,
         3.05385551,   3.19926714,   3.35160265,   3.51119173,
         3.67837977,   3.85352859,   4.03701726,   4....
        23.64489413,  24.77076356,  25.95024211,  27.18588243,
        28.48035868,  29.8364724 ,  31.2571585 ,  32.74549163,
        34.30469286,  35.93813664,  37.64935807,  39.44206059,
        41.320124  ,  43.28761281,  45.34878508,  47.50810162,
        49.77023564,  52.14008288,  54.62277218,  57.22367659,
        59.94842503,  62.80291442,  65.79332247,  68.92612104,
        72.20809018,  75.64633276,  79.24828984,  83.02175681,
        86.97490026,  91.11627561,  95.45484567, 100.        ]))

In [21]:
ridge.alpha_

1.0

In [22]:
update_df(ridge, "m005_ALL_35-69.L2-1")

,Model,Train Score,Val Score,X Val Score,RMSE Train,RMSE Val
0,m005_ALL_35-69.LinReg,1.000000,0.882940,0.616316,8.404957e-15,1.206978
1,m005_ALL_35-69.L2-1,0.999965,0.881611,0.614525,8.404957e-15,1.206978


In [23]:
lasso.fit(Xs_train, y_train)

LassoCV(alphas=array([1.000e-03, 1.001e+00, 2.001e+00, 3.001e+00, 4.001e+00, 5.001e+00,
       6.001e+00, 7.001e+00, 8.001e+00, 9.001e+00]))

In [24]:
update_df(lasso, "m005_ALL_35-69.L1-1")

,Model,Train Score,Val Score,X Val Score,RMSE Train,RMSE Val
0,m005_ALL_35-69.LinReg,1.000000,0.882940,0.616316,8.404957e-15,1.206978
1,m005_ALL_35-69.L2-1,0.999965,0.881611,0.614525,8.404957e-15,1.206978
2,m005_ALL_35-69.L1-1,0.999998,0.999996,0.999285,8.404957e-15,1.206978


In [25]:
importance = np.abs(lasso.coef_)
features = X.columns
np.array(features)[importance>0]

array(['q022', 'q096', 'q102', 'q105', 'q121', 'q122', 'q126', 'q152',
       'd038', 'm008_MEDICALc', 'm010_NONMEDc', 'p027', 'p034', 'p042',
       'r002', 'r016', 'r026'], dtype=object)

In [26]:
# np.array(features)
# importance
feat_imp = pd.DataFrame({'features':features, 'importance':importance}, columns=['features','importance'])

It'd be nice to be able to pull the code descriptions without searching the whole sheet one at a time. 

In [27]:
with open('./data/CHNAME.txt', 'r') as file:
    data = file.read()
    rows = data.split('\n')

descriptions = pd.DataFrame(rows)

In [28]:
# descriptions.head()
# return every other row
text_df = descriptions.iloc[::2,:]
text_df['code'] = text_df[0].str.split(expand=True).iloc[:,0]
text_df.head()

/var/folders/2f/m4z0nfhn5px0p2mdppmmrlxr0000gn/T/ipykernel_45950/2593100514.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df['code'] = text_df[0].str.split(expand=True).iloc[:,0]


,0,code
0,M001 ALL0-4 mortality ALL CAUSES AGE 0-4 (...,M001
2,M002 ALL5-14 mortality ALL CAUSES AGE 5-14 ...,M002
4,M003 ALL15-34 mortality ALL CAUSES AGE 15-34...,M003
6,M004 ALL0-34 mortality ALL CAUSES AGE 0-34 ...,M004
8,M005 ALL35-69 mortality ALL CAUSES AGE 35-69...,M005


In [29]:
text_df.rename(columns={0:'description'}, inplace=True)

/var/folders/2f/m4z0nfhn5px0p2mdppmmrlxr0000gn/T/ipykernel_45950/360386991.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df.rename(columns={0:'description'}, inplace=True)


In [30]:
text_df['code'] = text_df['code'].str.lower()

/var/folders/2f/m4z0nfhn5px0p2mdppmmrlxr0000gn/T/ipykernel_45950/2186823998.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df['code'] = text_df['code'].str.lower()


In [31]:
text_df.head()

,description,code
0,M001 ALL0-4 mortality ALL CAUSES AGE 0-4 (...,m001
2,M002 ALL5-14 mortality ALL CAUSES AGE 5-14 ...,m002
4,M003 ALL15-34 mortality ALL CAUSES AGE 15-34...,m003
6,M004 ALL0-34 mortality ALL CAUSES AGE 0-34 ...,m004
8,M005 ALL35-69 mortality ALL CAUSES AGE 35-69...,m005


In [32]:
feat_imp = feat_imp.merge(text_df, left_on='features', right_on='code', how='left').drop(columns = ['code'])

In [33]:
pd.set_option('max_colwidth', 150) # so I can see the full descriptions

In [34]:
feat_imp[feat_imp['importance']>0]

,features,importance,description
3,q022,0.000505,Q022 dEDUCATED questionnaire PERCENTAGE WHO ARE WELL-EDUCATED
23,q096,0.000027,Q096 dMALARIA questionnaire PERCENTAGE WITH HISTORY OF MALARIA DIAGNOSIS
29,q102,0.000886,Q102 dPHLEGMw questionnaire PERCENTAGE WHO COUGH UP PHLEGM MOST MORNINGS IN WINTER
32,q105,0.000454,Q105 dPHLEGMyr questionnaire NUMBER OF YEARS TROUBLED BY PHLEGM (years)
40,q121,0.000025,Q121 dANTIBIOT questionnaire PERCENTAGE USED MAINLY WESTERN ANTIBIOTICS DURING PAST 6 MONTHS
41,q122,0.000499,Q122 dANTACID questionnaire PERCENTAGE USED MAINLY WESTERN ANTACIDS DURING PAST 6 MONTHS
45,q126,0.000855,Q126 dWTLOSS questionnaire PERCENTAGE WHO LOST WEIGHT DURING FOOD SHORTAGE
61,q152,0.000007,Q152 dWINE questionnaire PERCENTAGE WHO HAVE EVER DRUNK WINE 3 OR MORE DAYS A WEEK FOR 6 MONTHS
107,d038,0.000116,"D038 WHTFLOUR diet survey WHEAT FLOUR INTAKE (g/day/reference man, air-dry basis)"
127,m008_MEDICALc,2.879213,NaN


We don't want other mortality reasons to affect our predictors. We will remove them from the feature list moving forward. For now, let's see what happens without the nonmedical mortality column

In [35]:
clean_df['m008_MEDICALc'].describe()

count    66.000000
mean     12.991667
std       3.039937
min       8.010000
25%      10.467500
50%      13.040000
75%      14.940000
max      21.230000
Name: m008_MEDICALc, dtype: float64

In [36]:
drop_columns = ['county', 'sex', 'xiang','m010_NONMEDc','m005_ALL35_69']

In [37]:
X = clean_df.drop(columns = drop_columns)
y = clean_df['m005_ALL35_69']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33)

In [38]:
Xs_train = ss.fit_transform(X_train)
Xs_test = ss.transform(X_test)

In [39]:
lr.fit(Xs_train, y_train)
lasso.fit(Xs_train, y_train)
update_df(lasso, "m005_ALL_35-69.L1-2")

/Users/colby/miniforge3/envs/tfm1/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.310e-02, tolerance: 4.664e-02
  model = cd_fast.enet_coordinate_descent(


,Model,Train Score,Val Score,X Val Score,RMSE Train,RMSE Val
0,m005_ALL_35-69.LinReg,1.000000,0.882940,0.616316,8.404957e-15,1.206978
1,m005_ALL_35-69.L2-1,0.999965,0.881611,0.614525,8.404957e-15,1.206978
2,m005_ALL_35-69.L1-1,0.999998,0.999996,0.999285,8.404957e-15,1.206978
3,m005_ALL_35-69.L1-2,0.999989,0.988186,0.931555,5.548124e-15,1.264333


These initial models all hold the mortality columns to train off of. Since we want to predict these columns, let's remove them all from the training data entirely to see if we're truly leaking data.

In [40]:
# clean_df.columns.to_list()
 # All the mortality columns
 # 'm005_ALL35_69',
 # 'm008_MEDICALc',
 # 'm065_STROKEc',
 # 'm023_ALLCAc',
 # 'm059_ALLVASCc',
 # 'm028_OESOPHCAc',
 # 'm072_COPDc',
 # 'm010_NONMEDc',

In [41]:
features_dropped = ['county', 'sex', 'xiang','m010_NONMEDc',\
                    'm005_ALL35_69', 'm008_MEDICALc', 'm065_STROKEc',\
                   'm023_ALLCAc', 'm059_ALLVASCc', 'm028_OESOPHCAc', 'm072_COPDc',\
                   'm072_COPDc']

mortality_feat = ['m010_NONMEDc', 'm005_ALL35_69', 'm008_MEDICALc', 'm065_STROKEc',\
                   'm023_ALLCAc', 'm059_ALLVASCc', 'm028_OESOPHCAc', 'm072_COPDc']

In [42]:
# Creating one code block to take care of all the models in one go:
lr = LinearRegression()
ss = StandardScaler()
lasso = LassoCV(alphas = np.arange(0.001, 10, 1))
# imp_features df will contain all the corr. coef/weights from each feature
imp_features = pd.DataFrame()
# base code modified from Jahnavi's new_model function
# looping over each target
for i in mortality_feat:
    # suppressing the convergence warnings if any
    import warnings
    warnings.filterwarnings("ignore")

    X = clean_df.drop(columns = features_dropped)
    y = clean_df[i]
    # train test split ('X before y, train before test' -James 2022)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33)
    
    ss.fit(X_train)
    Xs_train = ss.transform(X_train)
    Xs_test = ss.transform(X_test)
    
    lr.fit(Xs_train, y_train)
    lasso.fit(Xs_train, y_train)
    
    mortal = i

    update_df(lasso, mortal)
    
    # appending df to get feature importance (corr. coef) out
    feature_imp = pd.Series(lasso.coef_, index = X.columns)
    feature_imp = feature_imp.sort_values(ascending=False)
    
    imp_features[i] = feature_imp


In [43]:
results

,Model,Train Score,Val Score,X Val Score,RMSE Train,RMSE Val
0,m005_ALL_35-69.LinReg,1.000000,0.882940,0.616316,8.404957e-15,1.206978
1,m005_ALL_35-69.L2-1,0.999965,0.881611,0.614525,8.404957e-15,1.206978
2,m005_ALL_35-69.L1-1,0.999998,0.999996,0.999285,8.404957e-15,1.206978
3,m005_ALL_35-69.L1-2,0.999989,0.988186,0.931555,5.548124e-15,1.264333
4,m010_NONMEDc,0.482118,0.169658,-0.124616,1.332323e-13,44.534195
5,m005_ALL35_69,0.999993,0.056338,0.070911,7.825686e-15,2.689775
6,m008_MEDICALc,0.999991,0.107569,0.160962,6.704399e-15,2.591834
7,m065_STROKEc,0.770332,0.290192,0.009490,2.591966e-13,130.003377
8,m023_ALLCAc,0.999981,-0.538021,-0.466490,5.398386e-15,1.862238
9,m059_ALLVASCc,0.999958,0.138958,-0.104030,3.350999e-15,1.742879


We suspected as much. There is definitely a data leakage between the mortality features and should be withheld. With the data we have, there is effectively no predictive value in our models generated but may offer insight to any correlative features with mortality. 

Will is exploring PCA in his EDA notebook

In [44]:
# imp_features['m010_NONMEDc'][imp_features['m010_NONMEDc'] != 0].to_frame().sort_values(by='m010_NONMEDc', ascending=False)
# Forming a new df to store scores with the desciptive codes

In [45]:
text_df.columns

Index(['description', 'code'], dtype='object')

In [46]:
imp_features.reset_index(inplace=True)

In [47]:
score_df = imp_features.merge(text_df, left_on='index', right_on='code', how='left').drop(columns = ['index'])

In [48]:
score_df.head(2)

,m010_NONMEDc,m005_ALL35_69,m008_MEDICALc,m065_STROKEc,m023_ALLCAc,m059_ALLVASCc,m028_OESOPHCAc,m072_COPDc,description,code
0,7.084762,0.000000,-0.0,0.00000,-0.000000,0.000000,-0.000000,0.0,Q153 dWINEday questionnaire CURRENT DAILY CONSUMPTION OF WINE (g per person),q153
1,5.215087,0.087319,0.0,-1.88745,0.056419,-0.047364,4.606957,0.0,Q023 c%NOSCHL questionnaire PERCENTAGE OF HOUSEHOLD HEADS ATTENDED NO SCHOOL,q023


In [71]:
pd.set_option('display.max_rows', 100) 
# To see all the descriptions we need. I'll filter this down to top 10 later.

In [92]:
# This filtering will show only non-zero scores; 
# showing the weights of what the lasso regression actually used and how.
score_df[['m005_ALL35_69', 'description', 'code']][np.abs(score_df['m005_ALL35_69']) >0.40]\
.sort_values(by='m005_ALL35_69', ascending=False)

,m005_ALL35_69,description,code
117,1.352670,Q099 dBRTHFAST questionnaire PERCENTAGE WHO SUFFER BREATHLESSNESS WHEN HURRYING OR WALKING UPHILL,q099
92,0.726593,P043 HBsAb plasma HEPATITIS B ANTI-SURFACE ANTIGEN ANTIBODY,p043
8,0.658357,"D044 SALTVEG diet survey DRIED AND SALT-PRESERVED VEGETABLE INTAKE (g/day/reference man, as-consumed basis)",d044
42,0.482376,"D042 LIGHTVEG diet survey LIGHT COLOURED VEGETABLE INTAKE (g/day/reference man, fresh weight)",d042
150,0.454607,Q171 dSALTVEG questionnaire DAYS PER YEAR EAT SALT PRESERVED VEGETABLES,q171
107,0.425345,Q119 dCHINAMED questionnaire PERCENTAGE USED CHINESE MEDICINE REGULARLY DURING PAST 6 MONTHS,q119
128,-0.410812,Q040 c%INCALC questionnaire PERCENTAGE OF 1989 HOUSEHOLD INCOME SPENT ON ALCOHOL,q040
25,-0.545873,P009 B-CAROT plasma BETA CAROTENE (ug/dL),p009
88,-0.673292,P038 PEPSIN plasma PEPSINOGEN I/II,p038
188,-0.683745,P027 Cu plasma COPPER (mg/dL),p027


In [70]:
# imp_features['m008_MEDICALc'][imp_features['m008_MEDICALc'] != 0].to_frame().sort_values(by='m008_MEDICALc', ascending=False)
score_df[['m008_MEDICALc', 'description', 'code']][np.abs(score_df['m008_MEDICALc']) > 0.35]\
.sort_values(by='m008_MEDICALc', ascending=False)

,m008_MEDICALc,description,code
117,1.446953,Q099 dBRTHFAST questionnaire PERCENTAGE WHO SUFFER BREATHLESSNESS WHEN HURRYING OR WALKING UPHILL,q099
92,0.578106,P043 HBsAb plasma HEPATITIS B ANTI-SURFACE ANTIGEN ANTIBODY,p043
8,0.488523,"D044 SALTVEG diet survey DRIED AND SALT-PRESERVED VEGETABLE INTAKE (g/day/reference man, as-consumed basis)",d044
42,0.484209,"D042 LIGHTVEG diet survey LIGHT COLOURED VEGETABLE INTAKE (g/day/reference man, fresh weight)",d042
113,0.420315,Q125 dFAMINDUR questionnaire TOTAL DURATION OF SEVERE FOOD SHORTAGES DURING THE LAST 30 YEARS (months),q125
119,0.416942,Q136 dMFCTCIG questionnaire PERCENTAGE WHO HAVE EVER SMOKED MANUFACTURED CIGARETTES DAILY FOR MORE THAN 6 MONTHS,q136
107,0.388985,Q119 dCHINAMED questionnaire PERCENTAGE USED CHINESE MEDICINE REGULARLY DURING PAST 6 MONTHS,q119
150,0.378863,Q171 dSALTVEG questionnaire DAYS PER YEAR EAT SALT PRESERVED VEGETABLES,q171
89,-0.354926,P039 THYROXINE plasma TOTAL THYROXINE (ug/dL),p039
28,-0.364767,"D051 POULTRY diet survey POULTRY INTAKE (g/day/reference man, as-consumed basis)",d051


In [61]:
imp_features['m065_STROKEc'][imp_features['m065_STROKEc'] != 0].to_frame().sort_values(by='m065_STROKEc', ascending=False)
score_df[['m065_STROKEc', 'description', 'code']][score_df['m065_STROKEc'] != 0]\
.sort_values(by='m065_STROKEc', ascending=False)

,m065_STROKEc,description,code
98,26.612120,Q100 dBRTHLEV questionnaire PERCENTAGE WHO SUFFER BREATHLESSNESS WHEN WALKING WITH OTHERS ON LEVEL GROUND,q100
182,17.366433,Q158 dWHEAT questionnaire DAILY CONSUMPTION OF WHEAT (g/day air-dry basis),q158
103,17.271656,Q106 dWHEEZE questionnaire PERCENTAGE WHOSE CHEST OFTEN SOUNDS WHEEZY,q106
5,12.302131,R024 20:2n6 red blood cell TOTAL LIPID EICOSADIENOIC ACID (20:2(6)) (% of total fatty acid by weight),r024
61,7.621683,R013 22:0 red blood cell TOTAL LIPID BEHENIC ACID (22:0) (% of total fatty acid by weight),r013
92,6.231466,P043 HBsAb plasma HEPATITIS B ANTI-SURFACE ANTIGEN ANTIBODY,p043
142,6.158920,Q137 dCIGCONS questionnaire CURRENT DAILY CONSUMPTION OF MANUFACTURED CIGARETTES (no. per person),q137
105,1.866571,Q109 dDBP questionnaire DIASTOLIC BLOOD PRESSURE (mm Hg),q109
60,0.316645,R012 20:0 red blood cell TOTAL LIPID ARACHIDIC ACID (20:0) (% of total fatty acid by weight),r012
117,0.217354,Q099 dBRTHFAST questionnaire PERCENTAGE WHO SUFFER BREATHLESSNESS WHEN HURRYING OR WALKING UPHILL,q099


In [72]:
# imp_features['m023_ALLCAc'][imp_features['m023_ALLCAc'] != 0].to_frame().sort_values(by='m023_ALLCAc', ascending=False)
score_df[['m023_ALLCAc', 'description', 'code']][np.abs(score_df['m023_ALLCAc']) > 0.30]\
.sort_values(by='m023_ALLCAc', ascending=False)

,m023_ALLCAc,description,code
117,0.683157,Q099 dBRTHFAST questionnaire PERCENTAGE WHO SUFFER BREATHLESSNESS WHEN HURRYING OR WALKING UPHILL,q099
113,0.576475,Q125 dFAMINDUR questionnaire TOTAL DURATION OF SEVERE FOOD SHORTAGES DURING THE LAST 30 YEARS (months),q125
24,0.483848,P008 A-CAROT plasma ALPHA CAROTENE (ug/dL),p008
163,0.475164,D013 VITE diet survey TOTAL VITAMIN E INTAKE (mg/day/reference man),d013
127,0.453919,Q039 cSUPINC89 questionnaire HOUSEHOLD SIDELINE AND BUSINESS INCOME FOR 1989 (Yuan),q039
21,0.366521,P006 ALBUMIN plasma ALBUMIN (g/dL) (non-pooled analysis),p006
164,0.335755,Q164 dOILFAT questionnaire DAILY CONSUMPTION OF OIL AND FAT (g/day),q164
58,0.328760,R010 16:0 red blood cell TOTAL LIPID PALMITIC ACID (16:0) (% of total fatty acid by weight),r010
183,0.302693,Q159 dMAIZE questionnaire DAILY CONSUMPTION OF MAIZE (g/day air-dry basis),q159
177,-0.308406,Q152 dWINE questionnaire PERCENTAGE WHO HAVE EVER DRUNK WINE 3 OR MORE DAYS A WEEK FOR 6 MONTHS,q152


In [77]:
# imp_features['m059_ALLVASCc'][imp_features['m059_ALLVASCc'] != 0].to_frame().sort_values(by='m059_ALLVASCc', ascending=False)
score_df[['m059_ALLVASCc', 'description', 'code']][np.abs(score_df['m059_ALLVASCc']) > 0.14]\
.sort_values(by='m059_ALLVASCc', ascending=False)

,m059_ALLVASCc,description,code
117,0.400425,Q099 dBRTHFAST questionnaire PERCENTAGE WHO SUFFER BREATHLESSNESS WHEN HURRYING OR WALKING UPHILL,q099
182,0.376016,Q158 dWHEAT questionnaire DAILY CONSUMPTION OF WHEAT (g/day air-dry basis),q158
42,0.315672,"D042 LIGHTVEG diet survey LIGHT COLOURED VEGETABLE INTAKE (g/day/reference man, fresh weight)",d042
92,0.229516,P043 HBsAb plasma HEPATITIS B ANTI-SURFACE ANTIGEN ANTIBODY,p043
125,0.221994,Q037 cAGINC89 questionnaire HOUSEHOLD AGRICULTURAL INCOME FOR 1989 (Yuan),q037
43,0.159212,"D043 GREENVEG diet survey GREEN VEGETABLE INTAKE (g/day/reference man, fresh weight)",d043
155,0.150553,Q176 dEGGS questionnaire DAYS PER YEAR EAT EGGS,q176
69,0.147163,R022 22:6n3 red blood cell TOTAL LIPID DOCOSAHEXAENOIC ACID (22:6(3)) (% of total fatty acid by weight),r022
98,0.142196,Q100 dBRTHLEV questionnaire PERCENTAGE WHO SUFFER BREATHLESSNESS WHEN WALKING WITH OTHERS ON LEVEL GROUND,q100
75,-0.149340,P031 Zn plasma ZINC (mg/dL),p031


In [93]:
# imp_features['m028_OESOPHCAc'][imp_features['m028_OESOPHCAc'] != 0].to_frame().sort_values(by='m028_OESOPHCAc', ascending=False)
score_df[['m028_OESOPHCAc', 'description', 'code']][np.abs(score_df['m028_OESOPHCAc']) > 5]\
.sort_values(by='m028_OESOPHCAc', ascending=False)

,m028_OESOPHCAc,description,code
91,27.256861,P042 HBsAg plasma HEPATITIS B SURFACE ANTIGEN,p042
24,15.778510,P008 A-CAROT plasma ALPHA CAROTENE (ug/dL),p008
163,12.768285,D013 VITE diet survey TOTAL VITAMIN E INTAKE (mg/day/reference man),d013
113,10.214094,Q125 dFAMINDUR questionnaire TOTAL DURATION OF SEVERE FOOD SHORTAGES DURING THE LAST 30 YEARS (months),q125
25,-5.616304,P009 B-CAROT plasma BETA CAROTENE (ug/dL),p009
71,-9.320177,R025 20:3n6 red blood cell TOTAL LIPID DI-HOMO-GAMMA-LINOLENIC ACID (20:3(6)),r025
88,-12.848072,P038 PEPSIN plasma PEPSINOGEN I/II,p038
174,-13.961973,Q149 dALCEVER questionnaire PERCENTAGE EVER DRUNK ALCOHOL 3 OR MORE DAYS A WEEK FOR MORE THAN 6 MONTHS,q149
185,-18.799187,R026 20:4n6 red blood cell TOTAL LIPID ARACHIDONIC ACID (20:4(6)) (% of total fatty acid by weight),r026
144,-35.031992,Q165 dSMOKFOOD questionnaire PERCENTAGE EVER EAT SMOKED FOOD,q165


In [94]:
# imp_features['m072_COPDc'][imp_features['m072_COPDc'] > 0].to_frame().sort_values(by='m072_COPDc', ascending=False)
score_df[['m072_COPDc', 'description', 'code']][np.abs(score_df['m072_COPDc']) >10]\
.sort_values(by='m072_COPDc', ascending=False)

,m072_COPDc,description,code
103,29.647219,Q106 dWHEEZE questionnaire PERCENTAGE WHOSE CHEST OFTEN SOUNDS WHEEZY,q106
116,20.350247,Q128 dSMOKE questionnaire PERCENTAGE WHO HAVE EVER SMOKED ANY FORM OF TOBACCO DAILY FOR MORE THAN 6 MONTHS,q128
5,18.650919,R024 20:2n6 red blood cell TOTAL LIPID EICOSADIENOIC ACID (20:2(6)) (% of total fatty acid by weight),r024
7,15.993840,D028 PLNTFOOD diet survey PLANT FOOD INTAKE (g/day/reference man),d028
18,15.980178,P010 G-CAROT plasma GAMMA CAROTENE (ug/dL),p010
42,14.556566,"D042 LIGHTVEG diet survey LIGHT COLOURED VEGETABLE INTAKE (g/day/reference man, fresh weight)",d042
40,13.283286,"D040 STCHTUBER diet survey STARCHY TUBER INTAKE (g/day/reference man, fresh weight)",d040
144,12.236443,Q165 dSMOKFOOD questionnaire PERCENTAGE EVER EAT SMOKED FOOD,q165
142,-10.747447,Q137 dCIGCONS questionnaire CURRENT DAILY CONSUMPTION OF MANUFACTURED CIGARETTES (no. per person),q137
122,-14.221034,Q022 dEDUCATED questionnaire PERCENTAGE WHO ARE WELL-EDUCATED,q022


In [66]:
# I have discovered eli5 package and installed it
# <BIAS> is the expected average score of the model
# eli5.show_weights(lasso, feature_names = X_train.columns.to_list(), top=-1)